# 텍스트 분석

### 카운트 기반 벡터화  
Bag of words
- 단어들에 대해 츨현 횟수를 부여하는 방법, 문맥&순서는 무시
- 'and':0' 'baseball:3' 
- 희소행렬 문제 발생

TF-IDF
- 빈도수 기반 텍스트 벡터화 방법
- (term frequency - inverse document frequency)
- 공통적으로 많이 존재하는 단어 - 벡터화 작게
- 한 문서에만 많은 단어 - 벡터화 크게
- 모든 문서에 흔한 단어 제거 가능

전처리
- 목적: 분석의 정확도를 높이기 위해 
- 전처리에서 수행하는 작업 6가지

워드클라우드
- 수치데이터 -> 가시화 (가시성을 높히는 기법)


In [2]:
!pip install matplotlib

In [3]:
!pip install wordcloud

In [5]:
import pandas as pd
import glob
import re
from functools import reduce
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import STOPWORDS, WordCloud
# 패키지별 용도는 ppt참조

In [7]:
all_files = glob.glob('8장_data/myCabinetExcelData*.xls') # 파일 병합
all_files #출력하여 내용 확인

['8장_data\\myCabinetExcelData (1).xls',
 '8장_data\\myCabinetExcelData (2).xls',
 '8장_data\\myCabinetExcelData (3).xls',
 '8장_data\\myCabinetExcelData (4).xls',
 '8장_data\\myCabinetExcelData (5).xls',
 '8장_data\\myCabinetExcelData (6).xls',
 '8장_data\\myCabinetExcelData (7).xls',
 '8장_data\\myCabinetExcelData (8).xls',
 '8장_data\\myCabinetExcelData (9).xls',
 '8장_data\\myCabinetExcelData.xls']

In [9]:
all_files_data = [] #저장할 리스트
for file in all_files:
    data_frame = pd.read_excel(file)
    all_files_data.append(data_frame)
all_files_data[0] #작업 내용 확인

,Unnamed: 0,번호,제목,저자,출판사,출판일
0,NaN,1,Call for Special Issue Papers: Big Data Analyt...,"Balamurugan, S.; Muthu, Bala Anand; Peng, Shen...",Mary Ann Leibert,2020
1,NaN,2,"Interview with Dr. Silvio Carta, Author of the...","Carta, Silvio",Mary Ann Leibert,2019
2,NaN,3,Online Analytical Processing for Business Inte...,"Patel, Jigna Ashish; Sharma, Priyanka",Mary Ann Leibert,2020
3,NaN,4,A Big Data Guide to Understanding Climate Chan...,unknown,Mary Ann Leibert,2014
4,NaN,5,Call for Papers: Special Issue on Big Data in ...,unknown,Mary Ann Leibert,2016
...,...,...,...,...,...,...
95,NaN,96,2016 Index IEEE Transactions on Big Data Vol. 2,unknown,unknown,2017
96,NaN,97,Distributed Feature Selection for Efficient Ec...,"Zhao, L.; Chen, Z.; Hu, Y.; Min, G.; Jiang, Z.",unknown,2018
97,NaN,98,PaloPro: a platform for knowledge extraction f...,"Makrynioti, Nantia; Grivas, Andreas; Sardianos...",Inderscience,2017
98,NaN,99,A Hierarchical Distributed Processing Framewor...,"Dong, L.; Lin, Z.; Liang, Y.; He, L.; Zhang, N...",unknown,2016
